In [ ]:
# anemia_streamlit_app.py
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2
from sklearn.preprocessing import MinMaxScaler

# Load model
model = tf.keras.models.load_model("hybrid_model.h5")

# Configure MinMaxScaler (same config used during training)
scaler = MinMaxScaler()
scaler.min_, scaler.scale_ = [0.0], [1.0]  # Replace if trained differently

IMG_SIZE = 224

# Title
st.set_page_config(page_title="Anemia Detection App", layout="centered")
st.title("🩸 Anemia Detection using Conjunctiva Image + HB Level")

# File uploader
uploaded_file = st.file_uploader("Upload Conjunctiva Image", type=["jpg", "jpeg", "png"])
hb_input = st.number_input("Enter Hemoglobin (HB) Value", min_value=0.0, max_value=20.0, step=0.1)

if uploaded_file is not None and hb_input > 0:
    # Display uploaded image
    st.image(uploaded_file, caption="Uploaded Image", use_column_width=True)

    # Preprocess image
    image = Image.open(uploaded_file).convert("RGB")
    image = image.resize((IMG_SIZE, IMG_SIZE))
    image_np = np.array(image) / 255.0
    image_input = np.expand_dims(image_np, axis=0)

    # Preprocess HB
    hb_scaled = scaler.transform([[hb_input]])

    # Make prediction
    prediction = model.predict({"img_input": image_input, "hb_input": hb_scaled})[0][0]
    label = "Anemic" if prediction >= 0.5 else "Non-Anemic"

    st.subheader("🔍 Prediction Result")
    st.write(f"**Label:** {label}")
    st.write(f"**Probability:** {prediction:.4f}")

    st.success(f"The model predicts this person is likely **{label}**.")

else:
    st.warning("Please upload an image and enter a valid HB value above.")
